In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from vllm import LLM, SamplingParams

model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"

# ✅ Enable RoPE Scaling for 64K context
ROPE_SCALING_FACTOR = 2  # Scales from 32K → 64K

# ✅ Quantization: Load in 8-bit to save VRAM
quant_config = BitsAndBytesConfig(load_in_8bit=True)

# ✅ Define CPU Offloading for Lower Layers
device_map = {
    "transformer.wte": "cpu",    # Move embeddings to CPU
    "transformer.h.0": "cpu",    # Move first few layers to CPU
    "transformer.h.1": "cpu",
    "transformer.h.2": "cuda",   # Keep higher layers in GPU
    "transformer.h.3": "cuda",
    "transformer.ln_f": "cuda"   # Keep final normalization on GPU
}

# ✅ Load Model with Paged Attention (vLLM)
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mixtral-8x7B-Instruct-v0.1",
    quantization_config=quant_config,
    attn_implementation="flash_attention_2",  # Enable Flash Attention
    rope_scaling={"type": "dynamic", "factor": ROPE_SCALING_FACTOR},  # Scale RoPE to 64K
    device_map=device_map
)

# ✅ Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")


INFO 03-20 19:46:59 [__init__.py:256] Automatically detected platform cuda.


ModuleNotFoundError: No module named 'vllm._C'

In [4]:
import vllm
print(vllm.__version__)

RuntimeError: Failed to import transformers.processing_utils because of the following error (look up to see its traceback):
numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())


2.5.1+cu121
True
